### Conda Environment

Using Kernel: `dan-dev-py312-r433`

-----

# Ahringer Serizay Preprocess

#### Stage the data

- Staging Data from **source_data** to **derived_data**

**Files Derived:**
> - ahringer_erizay_all_tissues.xlsx

-----
- **Helper functions**

In [1]:
# Helper functions

import pandas as pd
from pathlib import Path

def load_and_sort_rna_metrics(file_path):
    """
    Reads the 'RNA_metrics' sheet from an Excel file, drops the first two rows,
    sets the third row as the header, and sorts the DataFrame by the 'Annotation' column.

    """
    # Read the sheet without setting a header initially
    df_raw = pd.read_excel(file_path, sheet_name='RNA_metrics', header=None)
    
    # Drop the first two rows
    df_cleaned = df_raw.iloc[2:].copy()
    
    # Set the third row (originally row index 2) as the header
    df_cleaned.columns = df_raw.iloc[2]
    df_cleaned = df_cleaned.iloc[1:]  # Drop the header row now moved to column names
    
    # Reset index for cleanlines
    df_cleaned.reset_index(drop=True, inplace=True)
    
    # Sort by 'Annotation' column
    if 'Annotation' in df_cleaned.columns:
        df_sorted = df_cleaned.sort_values(by='Annotation')
    else:
        raise KeyError("The column 'Annotation' was not found in the sheet.")
    
    return df_sorted


def split_by_annotation_and_export(rna_metrics_df, filters, output_file):
    """
    Splits the input DataFrame by unique 'Annotation' values and writes each
    subset to an Excel sheet. Each sheet contains only the 'WBID' values
    under the column name 'Wormbase ID'.

    """
    
    subset_dict = {}
    for name, condition in filters.items():
        subset = rna_metrics_df[rna_metrics_df['Annotation'].apply(condition)]
        subset_dict[name] = subset['WBID'].reset_index(drop=True)
        

    # Write to Excel
    directory = Path(output_file).parent
    directory.mkdir(parents=True, exist_ok=True)
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        for annotation, df in subset_dict.items():
            # Sheet names cannot be longer than 31 characters or contain certain symbols
            safe_sheet_name = str(annotation)[:31].replace('/', '_')
            df.to_excel(writer, sheet_name=safe_sheet_name, index=False)

    return subset_dict

-----

- **Preprocess**

In [2]:
source_data_path = "../source_data/ahringer_serizay"
derived_data_path = "../derived_data/ahringer_serizay"

In [3]:

rna_metrics_df = load_and_sort_rna_metrics(f"{source_data_path}/Supplemental_Table_S2.xlsx")

filters = {
    'unclassified' : lambda s: s == 'Unclassified',
    'ubiquitous'   : lambda s: s == 'Ubiq.',
    'ubi_biased'   : lambda s: s == 'Ubiq.-Biased',
    'sperm'        : lambda s: s == 'Sperm',
    'soma'         : lambda s: s == 'Soma',
    'neurons'      : lambda s: s == 'Neurons',
    'muscle'       : lambda s: s == 'Muscle',
    'intestine'    : lambda s: s == 'Intest.',
    'hypoderm'     : lambda s: s == 'Hypod.',
    'germline'     : lambda s: s == 'Germline',    
    'ubi_all'      : lambda s: s.startswith('Ubi'),
    'neuron_all'   : lambda s: s.startswith('Neuron'),
    'intestine_all': lambda s: 'Intest' in s,
    'muscle_all'   : lambda s: 'Muscle' in s,
    'hypoderm_all' : lambda s: 'Hypod' in s,
    'germline_all' : lambda s: 'Germline' in s
}

annotation_dict = split_by_annotation_and_export(rna_metrics_df, filters, f"{derived_data_path}/ahringer_serizay_all_tissues.xlsx")